In [1]:
# pip install sqlalchemy
# conda install sqlalchemy
import sqlalchemy

In [2]:
# verificar a versão
sqlalchemy.__version__

'1.4.22'

# Conectando ao banco de dados

In [3]:
# criar a interface de conexão ao banco de dados
# configurar o echo como True vai mostrar os códigos SQL gerados
engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True)

# Declarando o mapeamento

In [4]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [5]:
from sqlalchemy import Column, Integer, String

In [6]:
class User(Base):
    __tablename__ = 'users' # obrigatório

    id = Column(Integer, primary_key=True) # obrigatório
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)

# Criar a tabela no banco de dados

In [7]:
# criar a tabela no banco de dados
Base.metadata.create_all(engine)

2021-10-13 12:01:05,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 12:01:05,173 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-10-13 12:01:05,176 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-13 12:01:05,178 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2021-10-13 12:01:05,180 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-13 12:01:05,181 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	nickname VARCHAR(50), 
	PRIMARY KEY (id)
)


2021-10-13 12:01:05,182 INFO sqlalchemy.engine.Engine [no key 0.00153s] ()
2021-10-13 12:01:05,183 INFO sqlalchemy.engine.Engine COMMIT


# Criar as instâncias da classe mapeada

In [8]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [9]:
ed_user.name

'ed'

In [10]:
ed_user.fullname

'Ed Jones'

In [11]:
str(ed_user.id)

'None'

# Criar uma sessão

In [12]:
from sqlalchemy.orm import sessionmaker

# criando uma sessão e atribuindo a engine criad anteriormente
Session = sessionmaker(bind=engine)

In [13]:
# criar um objeto da sessão
session = Session()

# Adicionar objetos (INSERT)

In [14]:
# persistindo o objeto criado na sessão
session.add(ed_user)

Até agora, dizemos que a instância está pendente; nenhum SQL ainda foi criado e o objeto ainda não foi representado por uma linha no banco de dados. A sessão só enviará o SQL para persistir o usuário assim que for necessário, usando um processo conhecido como **flush**. Se consultarmos no banco de dados por pelo o usuário criado, todas as informações pendentes serão liberadas primeiro e a consulta será emitida imediatamente.

In [15]:
# consultar por dados
our_user = session.query(User).filter_by(name='ed').first()

2021-10-13 12:01:05,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 12:01:05,949 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-10-13 12:01:05,949 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('ed', 'Ed Jones', 'edsnickname')
2021-10-13 12:01:05,953 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2021-10-13 12:01:05,953 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('ed', 1, 0)


In [16]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [17]:
our_user.name

'ed'

In [18]:
# criar e adicionar diversos objetos na nossa sessão
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary')])

In [19]:
# verificar as novas mudanças pendentes
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>])

# Modificar objetos (UPDATE)

In [20]:
# mudar os dados do nosso objeto já criado (UPDATE)
ed_user.nickname = 'eddie'

In [21]:
# verificar cache de mudanças
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [22]:
# verificar as novas mudanças
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>])

In [23]:
# confirmar as mudanças
session.commit()

2021-10-13 12:02:06,378 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2021-10-13 12:02:06,379 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('eddie', 1)
2021-10-13 12:02:06,380 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-10-13 12:02:06,380 INFO sqlalchemy.engine.Engine [cached since 60.43s ago] ('wendy', 'Wendy Williams', 'windy')
2021-10-13 12:02:06,381 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-10-13 12:02:06,381 INFO sqlalchemy.engine.Engine [cached since 60.43s ago] ('mary', 'Mary Contrary', 'mary')
2021-10-13 12:02:06,382 INFO sqlalchemy.engine.Engine COMMIT


# Consultar objetos (SELECT)

In [26]:
# selecionar todos os usuários ordenados pelo o id
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2021-10-13 12:06:12,050 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2021-10-13 12:06:12,054 INFO sqlalchemy.engine.Engine [cached since 55.04s ago] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary


In [27]:
# selecionar colunas específicas
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2021-10-13 12:07:13,032 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2021-10-13 12:07:13,033 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary


In [31]:
# filtrar por um atributo
for name in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2021-10-13 12:09:17,119 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2021-10-13 12:09:17,120 INFO sqlalchemy.engine.Engine [cached since 29.15s ago] ('Ed Jones',)
('ed',)


# Deletar objetos (DELETE)

In [43]:
user = session.query(User).filter_by(name='wendy').first()

2021-10-13 12:20:46,235 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2021-10-13 12:20:46,236 INFO sqlalchemy.engine.Engine [cached since 1180s ago] ('wendy', 1, 0)


In [44]:
user

<User(name='wendy', fullname='Wendy Williams', nickname='windy')>

In [45]:
# deletar um usuário
session.delete(user)

In [46]:
# confirmar as mudanças
session.commit()

2021-10-13 12:21:14,059 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2021-10-13 12:21:14,060 INFO sqlalchemy.engine.Engine [cached since 334.4s ago] (2,)
2021-10-13 12:21:14,061 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
# consultar por informações refentes ao usuário deletado
session.query(User).filter_by(name='wendy').count()

2021-10-13 12:21:21,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 12:21:21,105 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2021-10-13 12:21:21,106 INFO sqlalchemy.engine.Engine [cached since 341.5s ago] ('wendy',)


0

Você pode consultar mais operações usando o SQLAlchemy na documentação da biblioteca: https://docs.sqlalchemy.org/en/14/orm/tutorial.html

Próximas etapa: 
- aprender a construir relações entre objetos;
- adicionar informações em tabelas pré-existentes;
- configurar as operações em cascata.